# Загружаем таблицу с проекта РеформаЖКХ

In [8]:
import pandas as pd

In [13]:
url = {
        'krasnoyarsk_building' : 'https://www.reformagkh.ru/opendata/export/155',
        'krasnoyarsk_uo' : 'https://www.reformagkh.ru/opendata/export/70'
        }
storage_options = {'User-Agent': 'Mozilla/5.0'}

In [21]:

building_gkh = pd.read_csv(url['krasnoyarsk_building'], storage_options=storage_options, encoding='utf-8', sep=';', compression='zip')
uo_gkh = pd.read_csv(url['krasnoyarsk_uo'], storage_options=storage_options, encoding='utf-8', sep=';', compression='zip')

## Выделяем таблицы для отдельных городов

In [30]:
kras_building_gkh = building_gkh[building_gkh.formalname_city == 'Красноярск']
achinsk_building_gkh = building_gkh[building_gkh.formalname_city == 'Ачинск']
kansk_building_gkh = building_gkh[building_gkh.formalname_city == 'Канск']
zelenogorsk_building_gkh = building_gkh[building_gkh.formalname_city == 'Зеленогорск']
cities = {
    'Krasnoyarsk' : kras_building_gkh,
    'Achinsk' : achinsk_building_gkh,
    'Kansk' : kansk_building_gkh,
    'Zelenogorsk' : zelenogorsk_building_gkh
}

In [31]:
for city in cities:
    cities[city].to_csv('../data/gkh_base_' + city + '.csv')
    dup = pd.DataFrame(cities[city].duplicated(keep=False, subset=['houseguid']), columns=['dup'])
    dup.reset_index(inplace=True)
    cities[city].reset_index(inplace=True)
    dup_city_dup = cities[city].merge(dup, how='left', left_on='index', right_on='index')
    dup_city_dup[dup_city_dup.dup == True].sort_values(by=['houseguid','id'], ascending=[True, True]).to_csv('../data/gkh_duplicates_' + city + '.csv')

## Удаляем дубликаты по маске

Предварительно надо вручную открыть файл duplicates, добавить столбец `delete` и вписать значения `True` либо `False` - удалить, либо оставить запись.

In [39]:
duplicates_achinsk = pd.read_csv('../data/gkh_duplicates_Achinsk_delete_mask.csv', encoding='utf-8', sep=',')

gkh_clear_Achinsk = achinsk_building_gkh.merge(duplicates_achinsk[['id','delete']], how='left', left_on='id', right_on='id')
gkh_clear_Achinsk.fillna(value={'delete':False}, inplace=True)
gkh_clear_Achinsk = gkh_clear_Achinsk[gkh_clear_Achinsk.delete == False]
gkh_clear_Achinsk.drop(columns='delete', inplace=True)
gkh_clear_Achinsk.to_csv('../data/gkh_clear_Achinsk.csv')

In [40]:
duplicates_kansk = pd.read_csv('../data/gkh_duplicates_Achinsk_delete_mask.csv', encoding='utf-8', sep=',')

gkh_clear_Kansk = kansk_building_gkh.merge(duplicates_kansk[['id','delete']], how='left', left_on='id', right_on='id')
gkh_clear_Kansk.fillna(value={'delete':False}, inplace=True)
gkh_clear_Kansk = gkh_clear_Kansk[gkh_clear_Kansk.delete == False]
gkh_clear_Kansk.drop(columns='delete', inplace=True)
gkh_clear_Kansk.to_csv('../data/gkh_clear_Kansk.csv')

In [41]:
duplicates_zelenogorsk = pd.read_csv('../data/gkh_duplicates_Zelenogorsk_delete_mask.csv', encoding='utf-8', sep=',')

gkh_clear_Zelenogorsk =zelenogorsk_building_gkh.merge(duplicates_zelenogorsk[['id','delete']], how='left', left_on='id', right_on='id')
gkh_clear_Zelenogorsk.fillna(value={'delete':False}, inplace=True)
gkh_clear_Zelenogorsk = gkh_clear_Zelenogorsk[gkh_clear_Zelenogorsk.delete == False]
gkh_clear_Zelenogorsk.drop(columns='delete', inplace=True)
gkh_clear_Zelenogorsk.to_csv('../data/gkh_clear_Zelenogorsk.csv')